# Hierarchial Indexing

In [2]:
import pandas as pd 
import numpy as np

# A Multiply Indexed Series¶
Let's start by considering how we might represent two-dimensional data within a one-dimensional Series. For concreteness, we will consider a series of data where each point has a character and numerical key.

## The bad way
Suppose you would like to track data about states from two different years. Using the Pandas tools we've already covered, you might be tempted to simply use Python tuples as keys:

In [3]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

With this indexing scheme, you can straightforwardly index or slice the series based on this multiple index:

In [5]:
pop[('California', 2000):('Texas', 2000)]

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

But the convenience ends there. For example, if you need to select all values from 2010, you'll need to do some messy (and potentially slow) munging to make it happen:

In [ ]:
pop[[i for i in pop.index if i[1] == 2010]]

This produces the desired result, but is not as clean (or as efficient for large datasets) as the slicing syntax we've grown to love in Pandas.

## The Better Way: Pandas MultiIndex
Fortunately, Pandas provides a better way. Our tuple-based indexing is essentially a rudimentary multi-index, and the Pandas MultiIndex type gives us the type of operations we wish to have. We can create a multi-index from the tuples as follows:

In [7]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

Notice that the MultiIndex contains multiple levels of indexing–in this case, the state names and the years, as well as multiple labels for each data point which encode these levels.

If we re-index our series with this MultiIndex, we see the hierarchical representation of the data:

### So here basically you first cook the multi-index instance, then use than to re-index the original data. Notice the original columns of data do not change and:
Denote n-level indicies with I_1, I_2..., I_n, and # of indicies in i-th level with len(I_i), then the total number of rows N should be equal to \sum\limits_{i=1} I_{i}

In [16]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Here the first two columns of the Series representation show the multiple index values, while the third column shows the data. Notice that some entries are missing in the first column: in this multi-index representation, any blank entry indicates the same value as the line above it.

Now to access all data for which the second index is 2010, we can simply use the Pandas slicing notation:

In [17]:
pop.loc["California", 2000]

33871648

In [19]:
pop.loc["California":"New York"]

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [20]:
pop.loc[:, 2000]

California    33871648
New York      18976457
Texas         20851820
dtype: int64

## MultiIndex as extra dimension
You might notice something else here: we could easily have stored the same data using a simple DataFrame with index and column labels. In fact, Pandas is built with this equivalence in mind. The `unstack()` method will quickly convert a multiply indexed Series into a conventionally indexed DataFrame:

In [21]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


Naturally, the `stack()` method provides the opposite operation:

In [22]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Seeing this, you might wonder why would we would bother with hierarchical indexing at all. The reason is simple: just as we were able to use multi-indexing to represent two-dimensional data within a one-dimensional Series, we can also use it to represent data of three or more dimensions in a Series or DataFrame. Each extra level in a multi-index represents an extra dimension of data; taking advantage of this property gives us much more flexibility in the types of data we can represent. Concretely, we might want to add another column of demographic data for each state at each year (say, population under 18) ; with a MultiIndex this is as easy as adding another column to the DataFrame:

In [24]:
pop_df = pd.DataFrame({'total': pop, #You might wonder why this works. That's b/c  pop  is nothing but a multi-indexed Series instance
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

# Methods of MultiIndex Creation
The most straightforward way to construct a multiply indexed `Series` or `DataFrame` is to simply pass a list of two or more index arrays to the constructor. For example:

In [ ]:
# THIS IS IMPORTANT!!

In [27]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.786121  0.818152
  2  0.795138  0.401510
b 1  0.250241  0.368872
  2  0.175669  0.736403

## THIS WORKS FOR ME

In [30]:
df = pd.DataFrame(np.random.rand(4, 3),
                  # Note here the number of unique elements of the first element of <index>(i.e. the date array) is the number of index in the first level(which is 1 here)
                  index=[['2019-01-29', '2019-01-29', '2019-01-29', '2019-01-29'], ["夫", "保育園", "子供", "慣れ"]], 
                  columns=['pos_id', 'category_id', 'count'])
df

pos_id  category_id     count
2019-01-29 夫    0.373020     0.379360  0.812025
           保育園  0.210677     0.398733  0.806550
           子供   0.138457     0.961371  0.189465
           慣れ   0.464885     0.952545  0.015169

The work of creating the `MultiIndex` is done in the background.

Similarly, if you pass a dictionary with appropriate tuples as keys, Pandas will automatically recognize this and use a `MultiIndex` by default:

### SO ONE IDEA FOR MY word_freq DATA IS TO FIRST BUILD THE TUPLE-INDEXED DICTIONARY INTO A `Series` INSTANCE, THEN UNPACK. SO THE PROBLEM LEFT NOW IS  TO FIGURE OUT HOW TO UNPACK.

In [31]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

Nevertheless, it is sometimes useful to explicitly create a MultiIndex; we'll see a couple of these methods here.

## Explicit MultiIndex constructors
For more flexibility in how the index is constructed, you can instead use the class method constructors available in the `pd.MultiIndex`. For example, as we did before, you can construct the `MultiIndex` from a simple list of arrays giving the index values within each level:

In [34]:
mi = pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]]) # From list of arrays

In [36]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)]) # From tuples

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [38]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]]) # From Cartesian products. Input should be list-like.

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [35]:
pd.Series(np.random.rand(4), index=mi)

a  1    0.309108
   2    0.918383
b  1    0.162435
   2    0.258194
dtype: float64

***To imitate the internal representation***, similarly, you can construct the `MultiIndex` directly using its internal encoding by passing `levels` (a list of lists containing available index values for each level) and `labels` (a list of lists that reference these labels): 

### The `index` argument in `Series`/`Dataframe` constructor
Any of these objects can be passed as the `index` argument when creating a `Series` or `Dataframe`, or be passed to the `reindex` method of an existing `Series` or `DataFrame`

## MultiIndex level names
Sometimes it is convenient to name the levels of the `MultiIndex`. This can be accomplished by passing the `names` argument to any of the above `MultiIndex` constructors, or by setting the `names` attribute of the index after the fact:

In [43]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [47]:
pop.index.names = ['state', 'year'] # With more involved datasets, this can be a useful way to keep track of the meaning of various index values
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

## MultiIndex for columns
In a `DataFrame`, the rows and columns are completely symmetric, and just as the rows can have multiple levels of indices, the columns can have multiple levels as well. Consider the following, which is a mock-up of some (somewhat realistic) medical data:

In [57]:
# hierarchical indices and columns 
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]], names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']], names=["subject", "type"])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2]
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      37.3  37.0  37.5  36.2  37.4  36.7
     2      37.0  37.9  35.3  38.1  35.1  35.8
2014 1      38.8  37.1  36.3  35.7  38.6  36.2
     2      35.2  37.4  35.0  38.5  36.5  37.4

***Where multi-index shines***: Here we see where the multi-indexing for both rows and columns can come in very handy. This is fundamentally four-dimensional data, where the dimensions are the subject, the measurement type, the year, and the visit number. With this in place we can, for example, index the top-level column by the person's name and get a full `DataFrame` containing just that person's information:

In [69]:
health_data['Guido']

year  visit
2013  1        37.5
      2        35.3
2014  1        36.3
      2        35.0
Name: HR, dtype: float64

# Indexing and Slicing a MultiIndex
Indexing and slicing on a `MultiIndex` is designed to be intuitive, and it helps if you think about the indices as added dimensions. We'll first look at indexing multiply indexed `Series`, and then multiply-indexed `DataFrames`.

## Multiply indexed Series
Consider the multiply indexed Series of state populations we saw earlier:

In [70]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [77]:
pop['New York', 2000] # A not-so-familiar way. By specifying index iteratively, you can access the exactly value you want

18976457

The `MultiIndex` also supports partial indexing, or indexing just one of the levels in the index. The result is another `Series`, with the lower-level indices maintained:

In [78]:
pop['California'] 

year
2000    33871648
2010    37253956
dtype: int64

In [ ]:
pop.loc['California':'New York'] # Partial slicing

With sorted indices, partial indexing can be performed on lower levels by passing an empty slice in the first index:

In [79]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

## Multiply indexed DataFrames
A multiply indexed `DataFrame` behaves in a similar manner. Consider our toy medical `DataFrame` from before:

In [80]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      37.3  37.0  37.5  36.2  37.4  36.7
     2      37.0  37.9  35.3  38.1  35.1  35.8
2014 1      38.8  37.1  36.3  35.7  38.6  36.2
     2      35.2  37.4  35.0  38.5  36.5  37.4

Remember that columns are primary in a `DataFrame`, and the syntax used for multi indexed `Series` applies to the columns. For example, we can recover Guido's heart rate data with a simple operation:

In [81]:
health_data["Bob", "HR"]

year  visit
2013  1        37.3
      2        37.0
2014  1        38.8
      2        35.2
Name: (Bob, HR), dtype: float64

In [82]:
health_data.iloc[:2, :2] # Access the dataframe as if it is a multi-dimensional numpy array

subject      Bob      
type          HR  Temp
year visit            
2013 1      37.3  37.0
     2      37.0  37.9

In [83]:
health_data.loc[:, ('Bob', 'HR')] # Each individual index can be passed a tuple of multiple indices
#health_data.loc[:, :"Bob"]

year  visit
2013  1        37.3
      2        37.0
2014  1        38.8
      2        35.2
Name: (Bob, HR), dtype: float64

subject      Bob      
type          HR  Temp
year visit            
2013 1      37.3  37.0
     2      37.0  37.9
2014 1      38.8  37.1
     2      35.2  37.4

In [ ]:
health_data.loc[(:, 1), (:, 'HR')] # This does not work. We cannot do slicing within a tuple(i.e. slicing multiple indices with respect to an axis is not supported) 

***Use an `IndexSlice` object to get around this***

In [86]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,37.3,37.5,37.4
2014,1,38.8,36.3,38.6


# Rearranging Multi-Indices
One of the keys to working with multiply indexed data is knowing how to effectively transform the data. There are a number of operations that will preserve all the information in the dataset, but rearrange it for the purposes of various computations. We saw a brief example of this in the `stack()` and `unstack()` methods, but there are many more ways to finely control the rearrangement of data between hierarchical indices and columns, and we'll explore them here.

## Sorted and unsorted indices¶
Earlier, we briefly mentioned a caveat, but we should emphasize it more here. *Many of the `MultiIndex` slicing operations will fail if the index is not sorted.* Let's take a look at this here.

We'll start by creating some simple multiply indexed data where the indices are *not lexographically* sorted:

In [101]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.randn(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1     -0.840437
      2      2.062516
c     1      0.118347
      2      1.499780
b     1     -0.482915
      2     -0.033361
dtype: float64

***If we try to take a partial slice of this index, it will result in an error:***

In [102]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


In [103]:
data



char  int
a     1     -0.840437
      2      2.062516
c     1      0.118347
      2      1.499780
b     1     -0.482915
      2     -0.033361
dtype: float64

In [104]:
data.sort_index()

char  int
a     1     -0.840437
      2      2.062516
b     1     -0.482915
      2     -0.033361
c     1      0.118347
      2      1.499780
dtype: float64

## Stacking and unstacking indices
As we saw briefly before, it is possible to convert a dataset from a stacked multi-index to a simple two-dimensional representation, optionally specifying the level to use:

In [105]:
pop.unstack(level=0) # Unstack the first level; Rearrange it to be the columns

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [106]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


## Index setting and resetting (VERY USEFUL FOR MY SCENARIO)

Another way to rearrange hierarchical data is to turn the index labels into columns; this can be accomplished with the `reset_index` method. Calling this on the population dictionary will result in a `DataFrame` with a *state* and *year* column holding the information that was formerly in the index. For clarity, we can optionally specify the name of the data for the column representation:

In [107]:
pop_flat = pop.reset_index(name="population")
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


Often when working with data in the real world, the raw input data looks like this and it's useful to build a `MultiIndex` from the column values. This can be done with the `set_index` method of the `DataFrame`, which returns a multiply indexed `DataFrame`:

In [118]:
pop_back0 = pop_flat.set_index(['state', 'year'])
pop_back0

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

# Data Aggregations on Multi-Indices
We've previously seen that Pandas has built-in data aggregation methods, such as `mean()`, `sum()`, and `max()`. For hierarchically indexed data, these can be passed a `level` parameter that controls which subset of the data the aggregate is computed on.

For example, let's return to our health data:

In [119]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      37.3  37.0  37.5  36.2  37.4  36.7
     2      37.0  37.9  35.3  38.1  35.1  35.8
2014 1      38.8  37.1  36.3  35.7  38.6  36.2
     2      35.2  37.4  35.0  38.5  36.5  37.4

Perhaps we'd like to average-out the measurements in the two visits each year. We can do this by naming the index level we'd like to explore, in this case the year:

In [137]:
data_mean = health_data.groupby(level="year").mean()
data_mean

subject    Bob         Guido           Sue       
type        HR   Temp     HR   Temp     HR   Temp
year                                             
2013     37.15  37.45  36.40  37.15  36.25  36.25
2014     37.00  37.25  35.65  37.10  37.55  36.80

By further making use of the `axis` keyword, we can take the mean among levels on the columns as well:

In [143]:
idx = pd.IndexSlice
health_data.groupby(level="year").mean().loc[2013, idx[:, "Temp"]].mean()

36.95

In [138]:
data_mean.groupby(axis=1, level="type").mean()

type,HR,Temp
year,,
2013,36.600000,36.95
2014,36.733333,37.05
